In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
from keras.layers.merge import _Merge

from keras.models import Model, Sequential
from keras import backend as K
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint 
from keras.utils import plot_model
from keras.initializers import RandomNormal
import keras

In [ ]:
data_gen = ImageDataGenerator(preprocessing_function=lambda x: (x.astype('float32')) / 255)
x_train = data_gen.flow_from_directory("/kaggle/input/celeba-dataset/img_align_celeba/"
                                        ,target_size = (256,256)
                                        ,batch_size = 64
                                        ,shuffle = True)

In [ ]:
# no. of batches
# image pixel values x_train[0][0] of 64 images (i.e. one batch)
# image label values x_train[0][1] of 64 images (i.e. one batch)
len(x_train), len(x_train[0]), x_train[0][0].shape

In [ ]:
# Images were scaled from -1 to 1. This would scale them from 0-1
plt.imshow((x_train[0][0][2]))

In [ ]:
keras.backend.clear_session()
weight_init = RandomNormal(mean=0., stddev=0.02)

discriminator_input = Input(shape= (256, 256, 3), name='discriminator_input')
x = discriminator_input

# First convolutional layer
x = Conv2D(filters= 64, kernel_size= 5, strides= 2, padding= 'same', name= 'discriminator_conv_1', 
           kernel_initializer= weight_init)(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Dropout(rate = 0.2)(x)

# Second convolutional layer
x = Conv2D(filters= 128, kernel_size= 5, strides= 2, padding= 'same', name= 'discriminator_conv_2', 
           kernel_initializer= weight_init)(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Dropout(rate = 0.2)(x)


# Third convolutional layer
x = Conv2D(filters= 256, kernel_size= 5, strides= 2, padding= 'same', name= 'discriminator_conv_3', 
           kernel_initializer= weight_init)(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Dropout(rate = 0.2)(x)


# Fourth convolutional layer
x = Conv2D(filters= 512, kernel_size= 5, strides= 2, padding= 'same', name= 'discriminator_conv_4', 
           kernel_initializer= weight_init)(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Dropout(rate = 0.2)(x)


x = Flatten()(x)

# discriminator_output = Dense(1, activation= None, kernel_initializer= weight_init)(x)
discriminator_output = Dense(1, activation= 'sigmoid', kernel_initializer= weight_init)(x)
discriminator = Model(discriminator_input, discriminator_output)

In [ ]:
discriminator.summary()

In [ ]:
generator_input = Input(shape=(100,), name='generator_input')
x = generator_input
x = Dense(np.prod(131072), kernel_initializer= weight_init)(x)
x = BatchNormalization(momentum = 0.9)(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Reshape((16, 16, 512))(x)

x = UpSampling2D()(x)
x = Conv2D(filters= 512, kernel_size= 5, padding= 'same', strides= 1, name = 'generator_conv_1', 
            kernel_initializer= weight_init)(x)
# x = Conv2DTranspose(filters= 256, kernel_size= 5, padding= 'same', strides= 2, name = 'generator_conv_1', 
#                     kernel_initializer= weight_init)(x)
x = BatchNormalization(momentum = 0.9)(x)
x = LeakyReLU(alpha = 0.2)(x)

x = UpSampling2D()(x)
x = Conv2D(filters= 256, kernel_size= 5, padding= 'same', strides= 1, name = 'generator_conv_2', 
            kernel_initializer= weight_init)(x)
# x = Conv2DTranspose(filters= 128, kernel_size= 5, padding= 'same', strides= 2, name = 'generator_conv_2', 
#                     kernel_initializer= weight_init)(x)
x = BatchNormalization(momentum = 0.9)(x)
x = LeakyReLU(alpha = 0.2)(x)

x = UpSampling2D()(x)
x = Conv2D(filters= 64, kernel_size= 5, padding= 'same', strides= 1, name = 'generator_conv_3', 
            kernel_initializer= weight_init)(x)
# x = Conv2DTranspose(filters= 64, kernel_size= 5, padding= 'same', strides= 2, name = 'generator_conv_3',
#                     kernel_initializer= weight_init)(x)
x = BatchNormalization(momentum = 0.9)(x)
x = LeakyReLU(alpha = 0.2)(x)


x = Conv2DTranspose(filters= 3, kernel_size= 5, padding= 'same', strides= 2, name = 'generator_conv_4',
                    kernel_initializer= weight_init)(x)
x = Activation('sigmoid')(x)

generator_output = x
generator = Model(generator_input, generator_output)

In [ ]:
generator.summary()

In [ ]:
import keras.backend as K

def wasserstein(y_true, y_pred):
    return - K.mean(y_true * y_pred)

# freezing weights of generator
generator.trainable = False
for layer in generator.layers:
    layer.trainable = False
    
discriminator.compile(optimizer= RMSprop(lr= 0.0008, decay=6e-8), 
                      loss= 'binary_crossentropy')

discriminator.trainable = False
for layer in discriminator.layers:
    layer.trainable = False
    
generator.trainable = True
for layer in generator.layers:
    layer.trainable = True
    
model_input = Input(shape=(100,), name='model_input')
model_output = discriminator(generator(model_input))

model = Model(model_input, model_output)

model.compile(optimizer= RMSprop(lr=0.0004, decay=3e-8),
              loss= 'binary_crossentropy')

discriminator.trainable = True
for layer in discriminator.layers:
    layer.trainable = True

In [ ]:
def train_discriminator(x_train, batch_size):

    valid = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))

    true_imgs = next(x_train)[0]
    if true_imgs.shape[0] != batch_size:
        true_imgs = next(x_train)[0]

    noise = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(true_imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * (d_loss_real + d_loss_fake)

    return [d_loss, d_loss_real, d_loss_fake]

def train_generator(batch_size):
    valid = np.ones((batch_size,1))
    noise = np.random.normal(0, 1, (batch_size, 100))
    return model.train_on_batch(noise, valid)

In [ ]:
import warnings
warnings.filterwarnings("ignore");

epoch = 0
d_losses = []
g_losses = []

for epoch in range(0, 3000):
    d_loss = train_discriminator(x_train, 64)
    g_loss = train_generator(64)
    
    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if epoch%100 == 0:
        print ("%d [D loss: (%.3f)(R %.3f, F %.3f)]  [G loss: %.3f] " % (epoch, d_loss[0], d_loss[1], d_loss[2], g_loss))
        
    epoch += 1

In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in d_losses], color='black', linewidth=0.25)

plt.plot([x[1] for x in d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in d_losses], color='red', linewidth=0.25)
plt.plot(g_losses, color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.show()

In [ ]:
noise = np.random.normal(0, 1, (20, 100))
gen_imgs = generator.predict(noise)
for i in gen_imgs:
    plt.figure()
    plt.imshow(i.reshape(256, 256, 3))
    plt.show();

In [ ]:
import warnings
warnings.filterwarnings("ignore");

epoch = 0
d_losses = []
g_losses = []

for epoch in range(0, 3000):
    d_loss = train_discriminator(x_train, 64)
    g_loss = train_generator(64)
    
    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if epoch%100 == 0:
        print ("%d [D loss: (%.3f)(R %.3f, F %.3f)]  [G loss: %.3f] " % (epoch, d_loss[0], d_loss[1], d_loss[2], g_loss))
        
    epoch += 1

In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in d_losses], color='black', linewidth=0.25)

plt.plot([x[1] for x in d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in d_losses], color='red', linewidth=0.25)
plt.plot(g_losses, color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.show()

In [ ]:
noise = np.random.normal(0, 1, (20, 100))
gen_imgs = generator.predict(noise)
for i in gen_imgs:
    plt.figure()
    plt.imshow(i.reshape(256, 256, 3))
    plt.show();